In [1]:
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
CURRENT_DIR = os.getcwd()
while not CURRENT_DIR.endswith("flight-pipeline") and CURRENT_DIR != "/":
    CURRENT_DIR = os.path.dirname(CURRENT_DIR)
os.chdir(CURRENT_DIR)
print("Working directory set to:", os.getcwd())

Working directory set to: /Users/kaancakir/data_projects_local/flight-pipeline


In [3]:
DATA_DIR = os.path.join(CURRENT_DIR, "data/processed/flights")
OUT_DIR = os.path.join(CURRENT_DIR, "data/analysis")
os.makedirs(OUT_DIR, exist_ok=True)

In [4]:
files = [f for f in os.listdir(DATA_DIR) if f.startswith("processed_flights_raw_") and f.endswith(".parquet")]

if not files:
    raise FileNotFoundError(f"No matching parquet files found in: {DATA_DIR}")

parquet_path = os.path.join(DATA_DIR, files[0])
print(f"Reading file: {parquet_path}")

Reading file: /Users/kaancakir/data_projects_local/flight-pipeline/data/processed/flights/processed_flights_raw_20251010_000138.parquet


In [5]:
con = duckdb.connect()
df = con.read_parquet(parquet_path)
con.register("flights", df)

In [6]:
q_top_countries = """
SELECT 
    origin_country,
    COUNT(*) AS flight_count,
    ROUND(AVG(velocity_kmh), 2) AS avg_velocity
FROM flights
GROUP BY origin_country
ORDER BY flight_count DESC
LIMIT 20;
"""
df_top_countries = con.sql(q_top_countries).df()
df_top_countries.to_csv(os.path.join(OUT_DIR, "top_countries.csv"), index=False)

In [7]:
q_speed_dist = """
SELECT 
    ROUND(velocity_kmh, -1) AS speed_bin,
    COUNT(*) AS count
FROM flights
WHERE velocity_kmh IS NOT NULL
GROUP BY speed_bin
ORDER BY speed_bin;
"""
df_speed_dist = con.sql(q_speed_dist).df()
df_speed_dist.to_csv(os.path.join(OUT_DIR, "speed_distribution.csv"), index=False)

In [8]:
q_altitude = """
SELECT 
    origin_country,
    ROUND(AVG(baro_altitude), 0) AS avg_altitude
FROM flights
WHERE baro_altitude IS NOT NULL
GROUP BY origin_country
ORDER BY avg_altitude DESC
LIMIT 15;
"""
df_altitude = con.sql(q_altitude).df()
df_altitude.to_csv(os.path.join(OUT_DIR, "avg_altitude.csv"), index=False)

In [9]:
q_corr = """
SELECT 
    CORR(velocity_kmh, baro_altitude) AS corr_speed_altitude
FROM flights
WHERE velocity_kmh IS NOT NULL AND baro_altitude IS NOT NULL;
"""
df_corr = con.sql(q_corr).df()
corr_value = df_corr.iloc[0, 0]
print(f"Correlation between speed and altitude: {corr_value:.3f}")

Correlation between speed and altitude: 0.916


In [10]:
q_fastest_countries = """
SELECT 
    origin_country,
    ROUND(AVG(velocity_kmh), 2) AS avg_velocity
FROM flights
WHERE velocity_kmh IS NOT NULL
GROUP BY origin_country
HAVING COUNT(*) > 30
ORDER BY avg_velocity DESC
LIMIT 10;
"""
df_fastest_countries = con.sql(q_fastest_countries).df()
df_fastest_countries.to_csv(os.path.join(OUT_DIR, "fastest_countries.csv"), index=False)


In [11]:
plt.figure(figsize=(10, 5))
plt.bar(df_top_countries["origin_country"], df_top_countries["flight_count"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 20 Countries by Flight Count")
plt.xlabel("Country")
plt.ylabel("Number of Flights")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "top_countries.png"))
plt.close()

In [12]:
plt.figure(figsize=(10, 5))
plt.plot(df_speed_dist["speed_bin"], df_speed_dist["count"], marker="o")
plt.title("Speed Distribution of Flights (km/h)")
plt.xlabel("Speed (km/h)")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "speed_distribution.png"))
plt.close()

In [13]:
plt.figure(figsize=(10, 5))
plt.bar(df_altitude["origin_country"], df_altitude["avg_altitude"])
plt.xticks(rotation=45, ha="right")
plt.title("Average Barometric Altitude by Country")
plt.xlabel("Country")
plt.ylabel("Average Altitude (m)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "avg_altitude.png"))
plt.close()

In [14]:
print("Analysis complete. Files saved to:", OUT_DIR)

Analysis complete. Files saved to: /Users/kaancakir/data_projects_local/flight-pipeline/data/analysis
